In [18]:
import numpy as np
import pandas as pd
import graph_tool.all as gt
import json
import matplotlib


language = "java"
project_name = "spring-boot"

In [19]:
with open('./../data/java-'+project_name+'.json') as f:
  data = json.load(f)

link_data = pd.DataFrame(data["links"])
link_data

,from,to,strength
0,org.springframework.boot.build,java.net.URI,1
1,org.springframework.boot.build,java.util.Collections,3
2,org.springframework.boot.build,java.util.HashMap,3
3,org.springframework.boot.build,java.util.Map,5
4,org.springframework.boot.build,org.asciidoctor.gradle.jvm.AbstractAsciidoctor...,1
...,...,...,...
29044,smoketest.xml,smoketest.xml.service.OtherService,1
29045,smoketest.xml,org.springframework.boot.test.context.SpringBo...,1
29046,smoketest.xml,org.springframework.context.annotation.Bean,1
29047,smoketest.xml,org.springframework.context.annotation.Configu...,1


In [20]:
g = gt.Graph()

v_list = {}
v_prop_list = []
v_prop = g.new_vertex_property("string")
for i, row in link_data.iterrows():
  if row['from'] not in v_list:
    v_list[row['from']] = g.add_vertex()
    v_prop[v_list[row['from']]] = row['from']
    v_prop_list.append(row['from'])
  if row['to'] not in v_list:
    v_list[row['to']] = g.add_vertex()
    v_prop[v_list[row['to']]] = row['to']
    v_prop_list.append(row['to'])

for i, row in link_data.iterrows():
  g.add_edge(v_list[row['from']], v_list[row['to']])

p = gt.sfdp_layout(g)
deg = g.degree_property_map("total")

In [21]:
import graph_tool.centrality as gtcentrality

In [22]:
page_rank_result = gtcentrality.pagerank(g)
p = gt.sfdp_layout(g, r=100)

# Stretch the layout somewhat
for v in g.vertices():
    p[v][0] *= 10000
    p[v][1] *= 10000

gt.graph_draw(g, vertex_fill_color=page_rank_result, vertex_text=v_prop,vertex_text_position = 0,
              pos=p,
              vertex_size=gt.prop_to_size(page_rank_result),
              vorder=page_rank_result, vcmap=matplotlib.cm.gist_heat,
              output="page-rank-result-java-"+project_name+".pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x1bc869450, at 0x1bc887040>

In [23]:
def show_page_rank_result():
  page_rank_result_array = page_rank_result.get_array()
  dic = {}
  for i in range(len(v_prop_list)):
    dic[v_prop_list[i]] = page_rank_result_array[i]

  sorted_dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
  for i in range(len(v_prop_list)):
    # print(sorted_dic[i])
    print("{0}位: {1} page_rank: {2}".format(i+1, sorted_dic[i][0], sorted_dic[i][1]))

show_page_rank_result()

1位: org.springframework.stereotype.Component page_rank: 0.00325011259830861
2位: org.springframework.context.annotation.Configuration page_rank: 0.0029683282400840426
3位: static org.assertj.core.api.Assertions.assertThat page_rank: 0.002651184623784381
4位: org.springframework.context.annotation.Bean page_rank: 0.00263361520381333
5位: org.junit.jupiter.api.Test page_rank: 0.001893444451892452
6位: org.springframework.boot.autoconfigure.SpringBootApplication page_rank: 0.0017751612231601246
7位: org.springframework.beans.factory.annotation.Autowired page_rank: 0.0012851599995306222
8位: org.springframework.web.bind.annotation.RestController page_rank: 0.0011904355070033144
9位: java.util.List page_rank: 0.0011153347029036326
10位: org.springframework.boot.context.properties.ConfigurationProperties page_rank: 0.0010785248365943311
11位: org.springframework.boot.test.context.SpringBootTest page_rank: 0.0009634951094449019
12位: org.springframework.data.repository.Repository page_rank: 0.0009595511